# Create Dataset

An example illustrating the creation of an image dataset using `jetcam`. 

The goal is to create a dataset containing images of people categorised into two labels based on whether they are looking into the camera or not. 

Run the cells of the notebook until the widgets appear. The widgets output cell may be opened in a new view in jupyter lab for a cleaner interface. 

To start capturing images, first review and update the camera settings (if required) and click on `Connect Camera` to instantiate the camera. From then on, the camera's view can be toggled using the `Live Camera` / `Stop` buttons.

The `Category` and `Session Label` inputs, together with the root folder specified in `Data Folder`, decide the destination folder for the captured images. The session label is optional. It allows the curator to cluster photos from different sessions into separate folders within each category. 

The `Save Image` button saves the current image when the camera is live, whereas it saves the image at the time of stopping video when the camera view is stopped.

The camera can be freed when the notebook kernel is shutdown.

In [ ]:
import os
from time import strftime, gmtime
import traitlets
from ipywidgets import Dropdown, Button, Text, Image, VBox, HBox, Label, ToggleButtons, IntText, GridspecLayout, HTML
from IPython.display import display
from pathlib import Path

import cv2

from jetcam.usb_camera import USBCamera
from jetcam.utils import bgr8_to_jpeg

In [ ]:
def grid(widgets, labels, **kwargs):
    rows = len(widgets)
    grid = GridspecLayout(rows, 2, **kwargs)
    for i in range(rows):
        grid[i, 0] = Label(labels[i])
        grid[i, 1] = widgets[i]
    return grid

### Create Widgets

In [ ]:
camera = None
API_PREFS = [('Any', cv2.CAP_ANY), 
             ('Gstreamer', cv2.CAP_GSTREAMER)]
CATEGORY_LABELS = [('Seeing', 'see'), ('Looking Away', 'away')]

In [ ]:
# Left panel
settings_header = HTML(value='<b>Camera Settings</b>')
device_widget = IntText(value=0)
api_pref_widget = Dropdown(options=API_PREFS)
height_widget = IntText(value=480)
width_widget = IntText(value=640)
fps_widget = IntText(value=30)
settings_widgets = [device_widget, api_pref_widget, height_widget, width_widget, fps_widget]
settings_labels = ['Device', 'API Preference', 'Capture Height', 'Capture Width', 'FPS']
settings_grid = grid(settings_widgets, settings_labels, width='450px')
connect_widget = Button(description='Connect Camera')

hr = HTML(value='<hr/>')

data_folder_widget = Text(value='data', 
                          description_tooltip='Folder where data will be stored')
category_widget = Dropdown(options=CATEGORY_LABELS)
session_widget = Text(description_tooltip='Optional folder to group images in a session')
data_info_grid = grid([data_folder_widget, category_widget, session_widget],
                      ['Data Folder', 'Category', 'Session Label'],
                      width='450px')
camera_toggle = ToggleButtons(options=[('Live Camera', True), ('Stop', False)], 
                              value=False)

left_panel = VBox([settings_header, settings_grid, connect_widget,
                   hr,
                   data_info_grid, camera_toggle])

In [ ]:
# Right panel
camera_view_header = HTML('<b>Camera View</b>')
camera_widget = Image(width=400, height=300, format='jpeg')

save_image_button = Button(description='Save Image', tooltip='Click photo')
right_panel = VBox([camera_widget, save_image_button])

all_panels = HBox([left_panel, right_panel])

In [ ]:
def connect_camera(c):
    global camera
    if camera is not None:
        camera.unobserve_all()
        camera.release_camera()
    camera = USBCamera(width=400,
                       height=300,
                       capture_height=height_widget.value,
                       capture_width=width_widget.value,
                       capture_fps=fps_widget.value,
                       capture_device=device_widget.value)
    traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
connect_widget.on_click(connect_camera)

def toggle_camera_view(change):
    global camera
    camera.running = change['new']
camera_toggle.observe(toggle_camera_view, names='value')

def create_image_path():
    path = Path(data_folder_widget.value) / category_widget.value
    if session_widget.value is not None:
        path = path / session_widget.value
    path.mkdir(parents=True, exist_ok=True)
    return path

def save_image(c):
    file_name = strftime('%y%m%d%H%M%S.jpg', gmtime())
    image_path = create_image_path() / file_name
    cv2.imwrite(str(image_path.absolute()), camera.value)
save_image_button.on_click(save_image)

### View 

In [ ]:
# Open the below output in a new view for a cleaner interface
display(all_panels)